In [1]:
!git clone https://github.com/brendenlake/omniglot.git
%cd omniglot/python
!unzip images_evaluation.zip
!unzip images_background.zip
%cd ../..

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_06.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_07.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_08.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_09.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_10.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_11.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_12.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_13.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_14.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_15.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_16.png  
  inflating: images_background/Malay_(Jawi_-_Arabic)/character28/0583_17.png  
  inflating: ima

In [2]:
import zipfile
from pathlib import Path
import glob
import os, sys

import numpy as np
import random

In [3]:
CUR_DIR = Path.cwd()
CUR_DIR

PosixPath('/content')

In [5]:
DATA_PATH = CUR_DIR / 'omniglot/python'
DATA_PATH

PosixPath('/content/omniglot/python')

In [6]:
BACKGROUND = DATA_PATH / 'images_background'
EVALUATION = DATA_PATH / 'images_evaluation'
print(BACKGROUND)
print(EVALUATION)

/content/omniglot/python/images_background
/content/omniglot/python/images_evaluation


In [7]:
backgrounds = [x for x in glob.glob(os.path.join(BACKGROUND, '*'))]
evaluations = [x for x in glob.glob(os.path.join(EVALUATION, '*'))]
print(len(backgrounds))
print(len(evaluations))

30
20


In [8]:
pick_10 = list(np.random.choice(evaluations, size=10, replace=False))
print(len(pick_10))

backgrounds += pick_10
print(len(backgrounds))

evaluations = [x for x in evaluations if x not in pick_10]
print(len(evaluations))

10
40
10


In [9]:
SEED = 42

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as dset
from torchvision import transforms
from PIL import Image

In [56]:
class TrainDataset(Dataset):
    def __init__(self, path_list, transform=None):
        super(TrainDataset, self).__init__()
        np.random.seed(SEED)
    
        self.transform = transform
        self.data, self.n_classes, self.len = self.load_data(path_list)
        
    def load_data(self, path_list):
        print('Start loading the data')
        data = {}
        idx, elements = 0, 0
        for path in path_list:
            #print(path)
            for char_path in os.listdir(path):
                if isinstance(char_path, bytes):
                    char_path = char_path.decode('utf-8')
                #print(char_path, type(char_path))
                data[idx] = []
                for img_path in os.listdir(os.path.join(path, char_path)):
                    img = os.path.join(path, char_path, img_path)
                    data[idx].append(Image.open(img).convert('L'))
                elements += len(data[idx])
                idx += 1
        print('Complete the loading')
        return data, idx, elements
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        label, img1, img2 = None, None, None
        
        # get two images from "same" class
        if idx % 2 == 1:
            label = 1 # same
            idx1 = random.randint(0, self.n_classes - 1)
            img1 = random.choice(self.data[idx1])
            img2 = random.choice(self.data[idx1])
    
        # get two images from "different" class
        else:
            label = 0 # different
            idx1 = random.randint(0, self.n_classes - 1)
            idx2 = random.randint(0, self.n_classes - 1)
            
            # idx1 and idx2 must be different
            while idx1 == idx2:
                idx2 = random.randint(0, self.n_classes - 1)
            img1 = random.choice(self.data[idx1])
            img2 = random.choice(self.data[idx2])
        
        # transform the data if exists
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.from_numpy(np.array([label], dtype=np.float32))
            

In [57]:
class TestDataset(Dataset):
    def __init__(self, path_list, transform=None, times=200, way=20):
        super(TestDataset, self).__init__()
        np.random.seed(SEED + 1)
    
        self.transform = transform
        self.times = times
        self.way = way
        self.img1 = None
        self.c1 = None
        self.data, self.n_classes, self.len = self.load_data(path_list)
        
    def load_data(self, path_list):
        print('Start loading the data')
        data = {}
        idx, elements = 0, 0
        for path in path_list:
            for char_path in os.listdir(path):
                data[idx] = []
                for img_path in os.listdir(os.path.join(path, char_path)):
                    img = os.path.join(path, char_path, img_path)
                    data[idx].append(Image.open(img).convert('L'))
                elements += len(data[idx])
                idx += 1
        print('Complete the loading')
        return data, idx, elements
    
    def __len__(self):
        return self.len
        #return self.times * self.way
    
    def __getitem__(self, idx):
        i = idx % self.way
        label = None
        if i == 0:
            self.c1 = random.randint(0, self.n_classes - 1)
            self.img1 = random.choice(self.data[self.c1])
            img2 = random.choice(self.data[self.c1])
        else:
            c2 = random.randint(0, self.n_classes - 1)
            while self.c1 == c2:
                c2 = random.randint(0, self.n_classes - 1)
            img2 = random.choice(self.data[c2])
            
        if self.transform:
            img1 = self.transform(self.img1)
            img2 = self.transform(img2)
            
        return img1, img2
            

In [58]:
transform_dataset = transforms.Compose([
    transforms.RandomAffine(10),
    transforms.ToTensor(),
])

train_set = TrainDataset(backgrounds, transform=transform_dataset)
test_set = TestDataset(evaluations, transform=transforms.ToTensor())

Start loading the data
Complete the loading
Start loading the data
Complete the loading


In [64]:
BATCH_SIZE = 32
TIMES = 400
WAY = 20

train_loader = DataLoader(train_set, 
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          )
test_loader = DataLoader(test_set,
                         batch_size=WAY,
                         shuffle=False,
                        )

In [65]:
import torch.nn as nn
import torch.nn.functional as F

class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(1, 64, 10),  # 64@96*96
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),  # 64@48*48
                nn.Conv2d(64, 128, 7),
                nn.ReLU(),    # 128@42*42
                nn.MaxPool2d(2),   # 128@21*21
                nn.Conv2d(128, 128, 4),
                nn.ReLU(), # 128@18*18
                nn.MaxPool2d(2), # 128@9*9
                nn.Conv2d(128, 256, 4),
                nn.ReLU(),   # 256@6*6
            )
        self.linear = nn.Sequential(nn.Linear(9216, 4096), nn.Sigmoid())
        self.out = nn.Linear(4096, 1)
        
    def forward_1(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x
    
    def forward(self, x1, x2):
        out1 = self.forward_1(x1)
        out2 = self.forward_1(x2)
        L1_dist = torch.abs(out1 - out2)
        out = self.out(L1_dist)
        return out

In [66]:
is_cuda = torch.cuda.is_available()
is_cuda

True

In [71]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import time
from fastprogress import progress_bar
import datetime


model = SiameseNet()
if is_cuda:
    model.cuda()
criterion = nn.BCEWithLogitsLoss(size_average=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 100
SHOW_EVERY = 50
TEST_EVERY = 50

train_loss = []
loss_val = 0

time_start = time.time()
for epoch in progress_bar(range(1, EPOCHS + 1)):
    train_loss = []
    time_start = time.time()
    for batch, (img1, img2, label) in enumerate(progress_bar(train_loader), 1):
        #print(batch)
        model.train()
        optimizer.zero_grad()
        if is_cuda:
            img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()
        output = model.forward(img1, img2)
        loss = criterion(output, label)
        train_loss.append(loss.item())
        loss_val += loss.item()
        
        loss.backward()
        optimizer.step()
        
        if batch % SHOW_EVERY == 0:
            print(f'[{epoch}][{batch}] Loss: {loss_val / SHOW_EVERY:.4f}, time_lapsed: {time.time() - time_start:.4f}')
            loss_val = 0
            time_start = time.time()
        
    # per-epoch
    model.eval()
    correct, wrong = 0, 0
    for _, (test1, test2) in enumerate(progress_bar(test_loader), 1):
        if is_cuda:
            test1, test2 = test1.cuda(), test2.cuda()
        output = model(test1, test2).data.cpu().numpy()
        pred = np.argmax(output)
        if pred == 0:
            correct += 1
        else:
            wrong += 1
    print('*' * 50)
    acc = correct / (correct + wrong)
    print(f'[{epoch}]| correct: {correct}, wrong: {wrong}, accuracy: {acc:.4f}')
    print('*' * 50)
    sns.lineplot(x=range(len(train_loss)), y=train_loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[1][50] Loss: 0.6433, time_lapsed: 9.5606
[1][100] Loss: 0.6222, time_lapsed: 9.4793
[1][150] Loss: 0.5981, time_lapsed: 9.4844
[1][200] Loss: 0.5953, time_lapsed: 9.4776
[1][250] Loss: 0.5632, time_lapsed: 9.4850
[1][300] Loss: 0.5527, time_lapsed: 9.4852
[1][350] Loss: 0.5271, time_lapsed: 9.4828
[1][400] Loss: 0.5136, time_lapsed: 9.4799
[1][450] Loss: 0.5109, time_lapsed: 9.4927
[1][500] Loss: 0.4958, time_lapsed: 9.4899
[1][550] Loss: 0.4616, time_lapsed: 9.4889
[1][600] Loss: 0.4643, time_lapsed: 9.4842
[1][650] Loss: 0.4541, time_lapsed: 9.4945
[1][700] Loss: 0.4722, time_lapsed: 9.4906
[1][750] Loss: 0.4318, time_lapsed: 9.5041
[1][800] Loss: 0.4266, time_lapsed: 9.5025


**************************************************
[1]| correct: 105, wrong: 219, accuracy: 0.3241
**************************************************


[2][50] Loss: 0.5130, time_lapsed: 9.4191
[2][100] Loss: 0.4162, time_lapsed: 9.5233
[2][150] Loss: 0.4439, time_lapsed: 9.5173
[2][200] Loss: 0.4078, time_lapsed: 9.5265
[2][250] Loss: 0.3957, time_lapsed: 9.5305
[2][300] Loss: 0.3773, time_lapsed: 9.5316
[2][350] Loss: 0.4001, time_lapsed: 9.5289
[2][400] Loss: 0.3870, time_lapsed: 9.5356
[2][450] Loss: 0.3401, time_lapsed: 9.5413
[2][500] Loss: 0.3286, time_lapsed: 9.5409
[2][550] Loss: 0.3463, time_lapsed: 9.5376
[2][600] Loss: 0.3299, time_lapsed: 9.5354
[2][650] Loss: 0.3682, time_lapsed: 9.5390
[2][700] Loss: 0.3395, time_lapsed: 9.5328
[2][750] Loss: 0.3374, time_lapsed: 9.5373
[2][800] Loss: 0.2887, time_lapsed: 9.5355


**************************************************
[2]| correct: 180, wrong: 144, accuracy: 0.5556
**************************************************


...

[42][50] Loss: 0.1181, time_lapsed: 9.4210
[42][100] Loss: 0.0892, time_lapsed: 9.5070
[42][150] Loss: 0.1041, time_lapsed: 9.5093
[42][200] Loss: 0.0906, time_lapsed: 9.5072
[42][250] Loss: 0.0871, time_lapsed: 9.5029
[42][300] Loss: 0.0953, time_lapsed: 9.5068
[42][350] Loss: 0.0942, time_lapsed: 9.5178
[42][400] Loss: 0.0892, time_lapsed: 9.5009
[42][450] Loss: 0.0821, time_lapsed: 9.5131
[42][500] Loss: 0.0932, time_lapsed: 9.5136
[42][550] Loss: 0.0926, time_lapsed: 9.5025
[42][600] Loss: 0.1130, time_lapsed: 9.5114
[42][650] Loss: 0.0695, time_lapsed: 9.5104
[42][700] Loss: 0.0828, time_lapsed: 9.5135
[42][750] Loss: 0.0860, time_lapsed: 9.5030
[42][800] Loss: 0.0976, time_lapsed: 9.5151


**************************************************
[42]| correct: 263, wrong: 61, accuracy: 0.8117
**************************************************


[43][50] Loss: 0.1055, time_lapsed: 9.4081
[43][100] Loss: 0.0836, time_lapsed: 9.5030
[43][150] Loss: 0.0994, time_lapsed: 9.5070
[43][200] Loss: 0.0716, time_lapsed: 9.5088
[43][250] Loss: 0.0712, time_lapsed: 9.5147
[43][300] Loss: 0.0918, time_lapsed: 9.5121
[43][350] Loss: 0.0894, time_lapsed: 9.5062
[43][400] Loss: 0.0841, time_lapsed: 9.5134
[43][450] Loss: 0.0821, time_lapsed: 9.5063
[43][500] Loss: 0.0887, time_lapsed: 9.5097
[43][550] Loss: 0.0778, time_lapsed: 9.5150
[43][600] Loss: 0.0963, time_lapsed: 9.5121
[43][650] Loss: 0.0946, time_lapsed: 9.5160
[43][700] Loss: 0.0797, time_lapsed: 9.5106
[43][750] Loss: 0.0800, time_lapsed: 9.5031
[43][800] Loss: 0.0933, time_lapsed: 9.5077


**************************************************
[43]| correct: 252, wrong: 72, accuracy: 0.7778
**************************************************


[44][50] Loss: 0.1035, time_lapsed: 9.4277
[44][100] Loss: 0.0843, time_lapsed: 9.5034
[44][150] Loss: 0.0881, time_lapsed: 9.5127
[44][200] Loss: 0.1043, time_lapsed: 9.5090
[44][250] Loss: 0.0859, time_lapsed: 9.5091
[44][300] Loss: 0.0813, time_lapsed: 9.5066
[44][350] Loss: 0.0763, time_lapsed: 9.5122
[44][400] Loss: 0.0969, time_lapsed: 9.5110
[44][450] Loss: 0.0870, time_lapsed: 9.5072
[44][500] Loss: 0.0831, time_lapsed: 9.5062


In [ ]:
SAVE_PATH = CUR_DIR / 'sample_data' / 'siamese.pt'
torch.save(model, SAVE_PATH)